# 공정성

## 목표

* 모델 데이터에서 나타날 수 있는 여러 유형의 편향에 대한 이해도 제고
* 모델을 학습시키기 전에 특성 데이터를 살펴보고 잠재적 데이터 편향 요인을 미리 파악
* 종합집계하는 대신 하위 그룹으로 묶어 모델 성능을 평가

## 개요

머신러닝(ML)에 원치 않는 편향이 발생할 수 있는 방식에 주목하면서 *공정성*을 염두에 두고 데이터세트를 살펴보고 분류자를 평가

공정성에 관한 ML 프로세스의 컨텍스트를 구성할 기회를 제공하는 **Fairness** 작업을 확인. 작업을 진행하는 동안 편향을 파악하고, 이러한 편향이 해결되지 않을 때 발생하는 모델 예측의 장기적인 영향을 고려

## 데이터세트 및 Prediction 작업 정보
서울대학교 보건 데이터셋을 활용

\- 원본 데이터세트에서 ML 공정성에 영향을 미칠 수 있는 field만을 임의 선택하여 학습에 사용

### Binary(이진) Features
*   `sex`: 성별
*   `cva`: 뇌졸중 과거력
*   `fcvayn`: 뇌졸중 가족력

### Numeric(수적) Features
*   `packyear`: 하루 흡연량(갑) X 흡연기간
*   `packyear`: 일주일간 음주 빈도
*   `exerfq`: 일주일간 운동한 총 일수

### Categorical(범주적) Features
*   `age`: 나이

### Prediction 작업
예측 작업은 조사 대상자의 성별을 예측하기 위해 실행

### Label
*   `sex`: 조사 대상자의 성별을 나타냄

## 공정성 지표

### 1. 균등 기회 (Equal Opportunity)

- definition: 보호 그룹과 보호되지 않은 그룹은 동일한 참긍정(True Positive)의 비율을 가져야 함
- `sex` field 에서 남녀 성별 확인을 위한 Prediction
- Category `cva` 중 Subgroup `0`(뇌졸중 과거력 있음) 입력에 따른 TPR과 Subgroup `1`(뇌졸중 과거력 없음) 입력에 따른 TPR이 같아야만 균등 기회(Equal Opportunity)를 만족

### 2. 균등 승률 (Equalized odds)

- definition: 보호된 그룹과 보호되지 않은 그룹은 참긍정(True Positive)과 오탐지(False Negative)에 대해 동일한 비율을 가져야 함
- `sex` field 에서 남녀 성별 확인을 위한 Prediction
- Category `cva` 중 Subgroup `0`(뇌졸중 과거력 있음) 입력에 따른 TPR, FNR과 Subgroup `1`(뇌졸중 과거력 없음) 입력에 따른 TPR, FNR이 같아야만 균등 승률(Equalized odds)을 만족

### 3. 인구통계패리티 (Demographic Parity)
- definition: 긍정적인 결과의 가능성은 개인이 보호된(예 : 여성) 그룹에 있는지 여부 에 관계없이 동일해야 함
- `sex` field 에서 남녀 성별 확인을 위한 Prediction
- Category `cva` 중 Subgroup `0`(뇌졸중 과거력 있음) 입력에 따른 TP+FP/TN+FN과 Subgroup `1`(뇌졸중 과거력 없음) 입력에 따른 TP+FP/TN+FN이 같아야만 인구통계패리티 (Demographic Parity)을 만족

## Dependency

In [1]:
import pandas as pd
import numpy as np
import math
import random
import tqdm
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from matplotlib import pyplot as plt
from matplotlib import rcParams

import seaborn as sns
import itertools

## Load dataset

### target
*   `sex`: 조사 대상자의 성별을 나타냄

### Subgroup
*   `cva`: 뇌졸중 과거력을 나타냄

In [2]:
train = pd.read_csv("before.csv")
train.fillna(0, inplace=True)

test = pd.read_csv('testset.csv')
test.fillna(0, inplace=True)

target = 'sex'
subgroup = 'cva'

col = list(train.columns)
except_target = col.copy()
except_target.remove(target)
lable=target
features = list(train.columns)
features.remove(lable)

In [3]:
y_train=train[lable].astype(int)
X_train=train[features].astype(int)


y_test=test[lable].astype(int)
X_test=test[features].astype(int)

CATEGORY  =  subgroup
SUBGROUP = 0 
X_test_a  = test.loc[test[CATEGORY] == SUBGROUP][features]
y_test_a  = test.loc[test[CATEGORY] == SUBGROUP][lable]

SUBGROUP = 1 
X_test_b  = test.loc[test[CATEGORY] == SUBGROUP][features]
y_test_b  = test.loc[test[CATEGORY] == SUBGROUP][lable]

## Fairness 작업 #1

* 보정 전 데이터셋를 사용하여 `SVC`, `LogisticRegression` 등 모델 학습
* 학습 결과에 따른 서브그룹별 TP, TN, FP, FN 도출 및 Confusion Matrix 시각화

In [4]:
def get_info(y_test, y_hat) : 
    
    tp = np.sum((y_test ==1) & (y_hat==1) )
    tn = np.sum((y_test ==0) & (y_hat==0) )
    fp = np.sum((y_test ==0) & (y_hat==1) )
    fn = np.sum((y_test ==1) & (y_hat==0) )
    
    accuracy = np.mean(np.equal(y_test,y_hat))
    
    return tp, tn, fp, fn, accuracy

In [5]:
#model = SVC(kernel = 'rbf')
#model = SVC(C=1)
model = LogisticRegression()
model.fit(X_train, y_train)

y_hat = model.predict(X_test_a)
tp_a, tn_a, fp_a, fn_a, accuracy = get_info(y_test_a, y_hat)
before_tpr_a = tp_a/(tp_a+fn_a)
before_fpr_a = fp_a/(fp_a+tn_a)
before_dp_a = (tn_a+fn_a)/(tp_a+fp_a)

y_hat = model.predict(X_test_b)
tp_b, tn_b, fp_b, fn_b, accuracy = get_info(y_test_b, y_hat)
before_tpr_b = tp_b/(tp_b+fn_b)
before_fpr_b = fp_b/(fp_b+tn_b)
before_dp_b = (tn_b+fn_b)/(tp_b+fp_b) 

print("before TPRA : " + str(before_tpr_a) + "/ before FPRA : " + str(before_fpr_a) + "/ before DFA : " + str(before_dp_a))
print("before TPRB : " + str(before_tpr_b) + "/ before FPRB : " + str(before_fpr_b) + "/ before DFB : " + str(before_dp_b))

before TPRA : 0.9326298701298701/ before FPRA : 0.8160406091370558/ before DFA : 0.14370284387695878
before TPRB : 0.8611111111111112/ before FPRB : 0.76/ before DFB : 0.23529411764705882


## Fairness 작업 #2

* Fairness 작업 #1에서 도출된 서브그룹 별 TPR 활용

In [6]:
def get_first_data( df, cur_col, tpra, tprb ) : 
    uniq = df[cur_col].unique()
    return_li = []
    for i in uniq : 
        return_li.append(df[df[cur_col]==i])

    arg_len = len(return_li)
    avg_ratio=abs(tpra-tprb)
    avg_fair=[]
    arg_sum = 0
    
    for i in range(arg_len) : 
        arg_sum+=len(return_li[i])
    arg_avg = arg_sum/arg_len
    
    for i in range(arg_len) : 
        if len(return_li[i])>arg_avg : 
            avg_fair.append(len(return_li[i])-(len(return_li[i])-arg_avg)*avg_ratio)
        else : avg_fair.append(len(return_li[i]))
            
    return return_li, avg_fair

In [7]:
def get_next_data( df, cur_col, ratio ) : 
    uniq = df[cur_col].unique()
    return_li = []
    for i in uniq : 
        return_li.append(df[df[cur_col]==i])
    arg_avg, avg_fair= get_avg(return_li, ratio)
    return return_li, avg_fair

def get_avg( arg, ratio ) : 
    arg_sum = 0
    arg_len = len(arg)
    avg_ratio=[]
    avg_fair=[]
    
    for i in range(arg_len) : 
        arg_sum+=(len(arg[i])*ratio)
    arg_avg = arg_sum/arg_len
        
    for i in range(arg_len) : 
        if len(arg[i])>arg_avg : 
            avg_fair.append(arg_avg)
        else : avg_fair.append(len(arg[i]))
            
    return arg_avg, avg_fair

In [8]:
def fairness(avg_fair, fair_data, col):

    tmp_fair_data=[]
    tmp_avg_li=[]

    for i,j in zip(fair_data,avg_fair) : 
        if j < len(i) : 
            ratio=1-((len(i)-j)/len(i))
        else : ratio=1
        #print(ratio)        
        return_li,avg = get_next_data(i, col, ratio)

        tmp_avg_li.append(avg)
        tmp_fair_data.append(return_li)

    tmp_fair_data=list(itertools.chain(*tmp_fair_data))
    #print(len(b))

    avg_fair=list(itertools.chain(*tmp_avg_li))
    
    return avg_fair, tmp_fair_data

In [9]:
fair_data, avg_fair = get_first_data(train, target, before_tpr_a, before_tpr_b)
for i in except_target : 
    avg_fair, fair_data = fairness(avg_fair, fair_data, i)

fair=pd.DataFrame(columns=col)
for i,j in zip(fair_data,avg_fair) :
    fair = pd.concat([i.sample(int(j)), fair])
    
y_fair=fair[lable].astype(int)
X_fair=fair[features].astype(int)

## Save Dataset

In [14]:
befor.to_csv("before10_14.csv", index=False)
after.to_csv("after10_14.csv", index=False)
test.to_csv("test10_14.csv", index=False)